In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [2]:
df=pd.read_csv('churn.csv')
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [3]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [7]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str)) 
    label_encoders[col] = le

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375,0,-0.439916,1,0,-1.277445,0,1,0,0,...,0,0,0,0,0,1,2,-1.160323,-0.398608,0
1,3962,1,-0.439916,0,0,0.066327,1,0,0,2,...,2,0,0,0,1,0,3,-0.259629,-0.948762,0
2,2564,1,-0.439916,0,0,-1.236724,1,0,0,2,...,0,0,0,0,0,1,3,-0.362660,-1.641883,1
3,5535,1,-0.439916,0,0,0.514251,0,1,0,2,...,2,2,0,0,1,0,0,-0.746535,-0.983710,0
4,6511,0,-0.439916,0,0,-1.236724,1,0,1,0,...,0,0,0,0,0,1,2,0.197365,-1.235224,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4853,1,-0.439916,1,1,-0.340876,1,2,0,2,...,2,2,2,2,1,1,3,0.665992,-0.879397,0
7039,1525,0,-0.439916,1,1,1.613701,1,2,1,0,...,2,0,2,2,1,1,1,1.277533,1.292099,0
7040,3367,0,-0.439916,1,1,-0.870241,0,1,0,2,...,0,0,0,0,0,1,2,-1.168632,-0.139680,0
7041,5934,1,2.273159,1,0,-1.155283,1,2,1,0,...,0,0,0,0,0,1,3,0.320338,-0.316534,1


In [11]:
x=df.drop(columns=['Churn'])
y=df['Churn']

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [13]:
bagging=BaggingClassifier(RandomForestClassifier(),n_estimators=50,random_state=42)
bagging.fit(x_train,y_train)
bagging_acc=accuracy_score(y_test,bagging.predict(x_test))

ada=AdaBoostClassifier(n_estimators=50,random_state=42)
ada.fit(x_train,y_train)
ada_acc=accuracy_score(y_test,ada.predict(x_test))

gb=GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,random_state=42)
gb.fit(x_train,y_train)
gb_acc=accuracy_score(y_test,gb.predict(x_test))

xgb=XGBClassifier(use_label_encoder=False,eval_metric='logloss',random_state=42)
xgb.fit(x_train,y_train)
xgb_acc=accuracy_score(y_test,xgb.predict(x_test))

C:\Users\CVR\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [14:40:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [15]:
print('Accuracy Score')
print("Bagging Accuracy:",bagging_acc)
print("AdaBoost Accuracy:", ada_acc)
print("Gradient Boosting Accuracy:", gb_acc)
print("XGBoost Accuracy:", xgb_acc)

Accuracy Score
Bagging Accuracy: 0.8035967818267865
AdaBoost Accuracy: 0.7917652626597255
Gradient Boosting Accuracy: 0.8017037387600567
XGBoost Accuracy: 0.7860861334595362


In [16]:
bagging_cls=classification_report(y_test,bagging.predict(x_test))
ada_cls=classification_report(y_test,ada.predict(x_test))
gb_cls=classification_report(y_test,gb.predict(x_test))
xgb_cls=classification_report(y_test,xgb.predict(x_test))

In [17]:
print('Classification Report')
print("Bagging :",bagging_cls)
print("AdaBoost :", ada_cls)
print("Gradient :", gb_cls)
print("XGBoost :", xgb_cls)

Classification Report
Bagging :               precision    recall  f1-score   support

           0       0.83      0.92      0.87      1539
           1       0.69      0.50      0.58       574

    accuracy                           0.80      2113
   macro avg       0.76      0.71      0.73      2113
weighted avg       0.79      0.80      0.79      2113

AdaBoost :               precision    recall  f1-score   support

           0       0.83      0.89      0.86      1539
           1       0.65      0.52      0.58       574

    accuracy                           0.79      2113
   macro avg       0.74      0.71      0.72      2113
weighted avg       0.78      0.79      0.78      2113

Gradient :               precision    recall  f1-score   support

           0       0.83      0.91      0.87      1539
           1       0.68      0.51      0.58       574

    accuracy                           0.80      2113
   macro avg       0.76      0.71      0.73      2113
weighted avg       0

In [18]:
bagging_mat=confusion_matrix(y_test,bagging.predict(x_test))
ada_mat=confusion_matrix(y_test,ada.predict(x_test))
gb_mat=confusion_matrix(y_test,gb.predict(x_test))
xgb_mat=confusion_matrix(y_test,xgb.predict(x_test))

In [21]:
print('Confusion Matrix')
print("Bagging :",bagging_mat)
print("AdaBoost :", ada_mat)
print("Gradient :", gb_mat)
print("XGBoost :", xgb_mat)

Confusion Matrix
Bagging : [[1411  128]
 [ 287  287]]
AdaBoost : [[1375  164]
 [ 276  298]]
Gradient : [[1404  135]
 [ 284  290]]
XGBoost : [[1359  180]
 [ 272  302]]
